## Entorno

In [5]:
#!pip install azureml-sdk

# Despliegue en la nube

In [1]:
from azureml.core import Workspace

ws = Workspace.create(name="Actividad_2",
                      subscription_id = "2d9980e2-949c-4940-90c6-ded37fd9868b",
                      resource_group = "__hw2__",
                      location = "centralindia") #centralindia

UserErrorException: UserErrorException:
	Message: You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 2d9980e2-949c-4940-90c6-ded37fd9868b subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscription_id='90eaa2d9-15dc-4bc3-9c4f-2c580923e179'), SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='a6c73a1f-e0ad-417d-862b-7d4070a6efb3')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to 2d9980e2-949c-4940-90c6-ded37fd9868b subscription, please check if it is in this tenant. All the subscriptions that you have access t

In [ ]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="modelo",
                                  workspace=ws)

Registering model modelo


In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [ ]:
import json

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('modelo')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)


    result = model.predict(data_dummies).tolist()
    result_sigmoid = sigmoid(result)
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=6)

service = Model.deploy(workspace=ws,
                       name='bankrupcy',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\aoran\AppData\Local\Temp\ipykernel_9232\950746086.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
service.wait_for_deployment()

Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 04ce4b94-f988-4a62-8914-b69813257f79
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 04ce4b94-f988-4a62-8914-b69813257f79
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config.",
  "details": [
    {
      "code": "ScoreInitRestart",
      "message": "Your scoring file's init() function restarts frequently. You can address the error by increasing the value of memory_gb in deployment_config."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: 04ce4b94-f988-4a62-8914-b69813257f79\nMore information can be found using

In [ ]:
scoring_uri = service.scoring_uri

In [ ]:
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()